In [1]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext line_profiler

from __future__ import division

import numpy as np
import glob
import matplotlib.pyplot as plt
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

In [2]:
# compute f_p statistic
def fpStat(psr, f0):
    """ 
    Computes the Fp-statistic as defined in Ellis, Siemens, Creighton (2012)
    
    :param psr: List of pulsar object instances
    :param f0: Gravitational wave frequency
    :return: Value of the Fp statistic evaluated at f0
    """

    fstat=0.
    npsr = len(psr)

    # define N vectors from Ellis et al, 2012 N_i=(x|A_i) for each pulsar
    N = np.zeros(2)
    # define M matrix M_ij=(A_i|A_j)
    M = np.zeros((2, 2))
    for ii,p in enumerate(psr):

        # Define A vector
        A = np.zeros((2, len(p.toas)))
        A[0,:] = 1./f0**(1./3.) * np.sin(2*np.pi*f0*p.toas)
        A[1,:] = 1./f0**(1./3.) * np.cos(2*np.pi*f0*p.toas)

        # define N vectors from Ellis et al, 2012 N_i=(x|A_i) for each pulsar
        N = np.array([np.dot(A[0,:], np.dot(p.invCov, p.residuals)), \
                      np.dot(A[1,:], np.dot(p.invCov, p.residuals))]) 
        
        # define M matrix M_ij=(A_i|A_j)
        for jj in range(2):
            for kk in range(2):
                M[jj,kk] = np.dot(A[jj,:], np.dot(p.invCov, A[kk,:]))
        
        # take inverse of M
        Minv = np.linalg.inv(M)
        fstat += 0.5 * np.dot(N, np.dot(Minv, N))

    # return F-statistic
    return fstat

In [3]:
# compute f_e-statistic
def feStat(psr, gwtheta, gwphi, f0):
    """ 
    Computes the F-statistic as defined in Ellis, Siemens, Creighton (2012)
    
    :param psr: List of pulsar object instances
    :param gwtheta: GW polar angle
    :param gwphi: GW azimuthal angle
    :param f0: Gravitational wave frequency
    :return: Value of the Fe statistic evaluated at gwtheta, phi, f0
    """
    
    npsr = len(psr)
    N = np.zeros(4)
    M = np.zeros((4,4))
    for ii, p in enumerate(psr):
        fplus, fcross, cosMu = createAntennaPatternFuncs(p, gwtheta, gwphi)

        # define A
        A = np.zeros((4, len(p.toas)))
        A[0,:] = fplus/f0**(1./3.) * np.sin(2*np.pi*f0*p.toas)
        A[1,:] = fplus/f0**(1./3.) * np.cos(2*np.pi*f0*p.toas)
        A[2,:] = fcross/f0**(1./3.) * np.sin(2*np.pi*f0*p.toas)
        A[3,:] = fcross/f0**(1./3.) * np.cos(2*np.pi*f0*p.toas)


        N += np.array([np.dot(A[0,:], np.dot(p.invCov, p.res)), \
                        np.dot(A[1,:], np.dot(p.invCov, p.res)), \
                        np.dot(A[2,:], np.dot(p.invCov, p.res)), \
                        np.dot(A[3,:], np.dot(p.invCov, p.res))]) 

        M += np.dot(A, np.dot(p.invCov, A.T))

    # inverse of M
    Minv = np.linalg.pinv(M)

    # Fe-statistic
    return 0.5 * np.dot(N, np.dot(Minv, N))

In [4]:
def createAntennaPatternFuncs(psr, gwtheta, gwphi):
    """
    Function to create pulsar antenna pattern functions as defined
    in Ellis, Siemens, and Creighton (2012).
    :param psr: pulsar object for single pulsar
    :param gwtheta: GW polar angle in radians
    :param gwphi: GW azimuthal angle in radians
    :return: (fplus, fcross, cosMu), where fplus and fcross
             are the plus and cross antenna pattern functions
             and cosMu is the cosine of the angle between the 
             pulsar and the GW source.
    """

    # use definition from Sesana et al 2010 and Ellis et al 2012
    m = np.array([-np.sin(gwphi), np.cos(gwphi), 0.0])
    n = np.array([-np.cos(gwtheta)*np.cos(gwphi), -np.cos(gwtheta)*np.sin(gwphi),\
                  np.sin(gwtheta)])
    omhat = np.array([-np.sin(gwtheta)*np.cos(gwphi), -np.sin(gwtheta)*np.sin(gwphi),\
                      -np.cos(gwtheta)])

    phat = np.array([np.sin(psr.theta)*np.cos(psr.phi), np.sin(psr.theta)*np.sin(psr.phi), \
                     np.cos(psr.theta)])

    fplus = 0.5 * (np.dot(m, phat)**2 - np.dot(n, phat)**2) / (1+np.dot(omhat, phat))
    fcross = (np.dot(m, phat)*np.dot(n, phat)) / (1 + np.dot(omhat, phat))
    cosMu = -np.dot(omhat, phat)

    return fplus, fcross, cosMu

In [5]:
datadir = enterprise.__path__[0] + '/datafiles/mdc_open1/'
#datadir = '~/Documents/Grad\ School/Research/enterprise/enterprise/datafiles'
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

In [22]:
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t)
    psrs.append(psr)

In [21]:
print(psrs[0].residuals)
print(len(psrs[0].toas))

[-2.61949243e-06 -2.30768375e-06 -1.91043621e-06 -1.87441510e-06
 -1.45311256e-06 -1.38749765e-06 -1.09011676e-06 -9.43985922e-07
 -8.78250976e-07 -7.64967995e-07 -5.45120466e-07 -3.92265082e-07
 -4.01810854e-07 -3.43540928e-07 -2.30470376e-07 -3.07846452e-07
 -2.43889243e-07 -3.27004794e-07 -7.02494461e-08 -6.01813389e-08
  1.71676462e-07  3.52073411e-07  6.91932668e-07  9.14128148e-07
  1.09731300e-06  1.59911385e-06  1.81857537e-06  1.90637141e-06
  2.04803756e-06  2.13960089e-06  2.13818234e-06  2.02010317e-06
  1.91892227e-06  1.83017811e-06  1.86189761e-06  1.71753654e-06
  1.62197032e-06  1.38048729e-06  1.41963837e-06  1.00268638e-06
  9.69274967e-07  9.33838787e-07  5.51357349e-07  6.12891358e-07
  5.46071571e-07  5.17916582e-07  2.27609980e-07  2.53671480e-07
  8.06939970e-08  1.21295671e-07  1.31952375e-07  1.81600209e-07
  3.49476955e-07  3.23414579e-07  1.55866106e-07  2.67220396e-07
  2.78480870e-07  2.40311339e-07  3.46497373e-07 -5.45645987e-08
 -8.38194787e-09 -2.89014

In [26]:
f_gw = 8e-8
fp = fpStat(psrs,f_gw)

AttributeError: 'Tempo2Pulsar' object has no attribute 'invCov'